In [1]:
import pandas as pd
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.models as models
from torch.utils.data import Dataset
import torchaudio
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

C:\ProgramData\Miniconda3\envs\voice\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


FileNotFoundError: Could not find module 'C:\ProgramData\Miniconda3\envs\voice\lib\site-packages\scipy\.libs\libbanded5x.3OIBJ6VWWPY6GDLEMSTXSIPCHHWASXGT.gfortran-win_amd64.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [ ]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from model.classifier.dataset import GenocidalClassifierDataset

In [3]:
train_set = GenocidalClassifierDataset('../data/datasets/outros/outros.csv',
                                       '../data/datasets/bolsoanta/bolsoanta.csv',
                                       resample_freq=22050, n_mels=64, split=.8)
test_set = GenocidalClassifierDataset('../data/datasets/outros/outros.csv',
                                      '../data/datasets/bolsoanta/bolsoanta.csv',
                                      resample_freq=22050, n_mels=64, split=-.2)

In [4]:
print(f"Train set size: {len(train_set)} - {len(train_set)*100/(len(train_set)+len(test_set)):2.0f}%")
print(f"Test set size: {len(test_set)} - {len(test_set)*100/(len(train_set)+len(test_set)):2.0f}%")

Train set size: 6746 - 80%
Test set size: 1686 - 20%


In [5]:
spec, label = train_set[0]

In [6]:
print(spec.shape)
print(label)

torch.Size([1, 64, 330])
0


In [7]:
label

0

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu

train_loader = torch.utils.data.DataLoader(train_set, batch_size = 128, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 128, shuffle = True, **kwargs)

In [10]:
model = models.resnet18(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Fix initial and final layers of the model

In [58]:
model = models.resnet18(pretrained=True)
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
model = models.resnet18(pretrained=True)
model.conv1=nn.Conv2d(1, model.conv1.out_channels, 
                      kernel_size=model.conv1.kernel_size[0], 
                      stride=model.conv1.stride[0], 
                      padding=model.conv1.padding[0])
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)
model.to(device)
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2):

In [12]:
adam = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)

In [52]:
loss = nn.BCEWithLogitsLoss()
def train(model, epoch, criterion, optimizer, dataloader, device):
    """Trains the model for 1 epoch

    Args:
        model: Pytorch model to be trained
        criterion: Loss function
        optimizer: Optimizer used in the training
        dataloader: Dataloader for training data
        device: Where the model is being trained
        epoch (int): Current epoch being trained
    """
    model.train()
    for batch_idx, (data, target) in enumerate(dataloader):
        optimizer.zero_grad()
        data = data.to(device)
        target = torch.unsqueeze(target.to(device), 1).to(torch.float32)
        output = model(data)
        loss = criterion(output, target)
        # Backprogate loss with gradients
        loss.backward()
        optimizer.step()
        # Print stats
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t\t Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(dataloader.dataset),
                100. * batch_idx / len(dataloader.dataset), loss))

In [53]:
train(model, 1, loss, adam, train_loader, device)

Train Epoch: 1 [0/6746 (0%)]		 Loss: 0.055768
Train Epoch: 1 [1280/6746 (0%)]		 Loss: 0.042211
Train Epoch: 1 [2560/6746 (0%)]		 Loss: 0.075158
Train Epoch: 1 [3840/6746 (0%)]		 Loss: 0.056409
Train Epoch: 1 [5120/6746 (1%)]		 Loss: 0.023238
Train Epoch: 1 [6400/6746 (1%)]		 Loss: 0.035489


In [54]:

def test_binary_classification(model, dataloader, device):
    model.eval()
    model.to(device)
    predictions = []
    labels = []
    for (data, target) in dataloader:
        data = data.to(device)
        target = torch.unsqueeze(target.to(device), 1).to(torch.float32)
        output = torch.sigmoid(model(data))
        pred = (output > .5)
        predictions.append(torch.squeeze(pred).cpu().numpy().astype(int))
        labels.append(torch.squeeze(target).cpu().numpy())
    # Show stats
    predictions = np.concatenate(predictions)
    labels = np.concatenate(labels)
    print(f'''\nTestset metrics ({len(dataloader.dataset)} samples)\n
             Accuracy: \t {accuracy_score(labels, predictions)*100:.2f}%\n
             F1-score: \t {f1_score(labels, predictions):.2f}''')

In [61]:
test_binary_classification(model, test_loader, device)


Testset metrics (1686 samples)

             Accuracy: 	 90.27%

             F1-score: 	 0.90


(array([0, 1, 1, ..., 1, 0, 1]),
 array([0., 1., 1., ..., 1., 0., 1.], dtype=float32))

## Hyperparameters

In [22]:
EPOCHS = 10
LOG_INTERVAL = 5

In [51]:
for epoch in range(1, EPOCHS):
    train(model, epoch, loss, adam)
    test(model)

Train Epoch: 1 [0/6746 (0%)]	Loss: 0.109747
Train Epoch: 1 [1280/6746 (19%)]	Loss: 0.051015
Train Epoch: 1 [2560/6746 (38%)]	Loss: 0.017187
Train Epoch: 1 [3840/6746 (57%)]	Loss: 0.076046
Train Epoch: 1 [5120/6746 (75%)]	Loss: 0.037383


KeyboardInterrupt: 

In [59]:
def load_model(path="model.pt"):
    model = models.resnet18(pretrained=True)
    model.conv1=nn.Conv2d(1, model.conv1.out_channels, 
                        kernel_size=model.conv1.kernel_size[0], 
                        stride=model.conv1.stride[0], 
                        padding=model.conv1.padding[0])
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 1)
    model.load_state_dict(torch.load(path))
    model.eval()
    return model
    

In [60]:
model = load_model()

In [42]:
import IPython.display as ipd
sample_no = test_set.data.iloc[4]
sample_yes = test_set.data.iloc[1600]

# Test
audio, spec_no = test_set._prepare_audio(sample_no.source)
print(f"Audio:{sample_no.source}")
print(f"Label:{sample_no.label}")
ipd.Audio(sample_no.source)

Audio:D:/ARQUIVOS PESSOAIS/GitHub/GenocidalVoice/data/datasets/outros/wav/003379.wav
Label:0


In [43]:
# Test
audio, spec_yes = test_set._prepare_audio(sample_yes.source)
print(f"Audio:{sample_yes.source}")
print(f"Label:{sample_yes.label}")
ipd.Audio(sample_yes.source)

Audio:D:/ARQUIVOS PESSOAIS/GitHub/GenocidalVoice/data/datasets/bolsoanta/wav/004132.wav
Label:1


In [44]:
model.to(device)
out = torch.sigmoid(model(spec_no.to(device).unsqueeze(0)))
print(f"Probability: {out.cpu().item() * 100:2.1f} %")

Probability: 0.0 %


In [45]:
model.to(device)
out = torch.sigmoid(model(spec_yes.to(device).unsqueeze(0)))
print(f"Probability: {out.cpu().item() * 100:2.1f} %")

Probability: 98.6 %
